In [1]:
import numpy as np
import re
import nltk
from sklearn.datasets import load_files
nltk.download('stopwords')
import pickle
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Marc\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
movie_data = load_files(r"C:\Users\Marc\Documents\MasterThesis\txt_sentoken")
movie_reviews, y = movie_data.data, movie_data.target

In [3]:
documents = []

TAG_RE = re.compile(r'<[^>]+>')

def remove_tags(text):
    return TAG_RE.sub('', text)

for sen in range(0, len(movie_reviews)):
    stringReview = movie_reviews[sen].decode()
    
    stringReview = stringReview.replace('\n','')
    
    # Removing html tags
    stringReview = remove_tags(stringReview)
    
     # Remove punctuations and numbers
    stringReview = re.sub('[^a-zA-Z]', ' ', stringReview)

    # Single character removal
    stringReview = re.sub(r"\s+[a-zA-Z]\s+", ' ', stringReview)

    # Removing multiple spaces
    stringReview = re.sub(r'\s+', ' ', stringReview)
    
    # Remove all the special characters
    document = re.sub(r'\W', ' ', stringReview)
    
    # Substituting multiple spaces with single space
    document = re.sub(r'\s+', ' ', document, flags=re.I)
    
    # Removing prefixed 'b'
    document = re.sub(r'^b\s+', '', document)
    
    # Converting to Lowercase
    document = document.lower()
    
    document = document.split()

    #space normalization
    document = ' '.join(document)
    
    #documents.append(document)
    documents.append(document)

In [4]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(max_features=1500, min_df=5, max_df=0.7, stop_words=stopwords.words('english'))
X = vectorizer.fit_transform(documents)

In [5]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidfconverter = TfidfTransformer()
X = tfidfconverter.fit_transform(X)

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [7]:
from sklearn.ensemble import RandomForestClassifier

classifier = RandomForestClassifier(n_estimators=1000, random_state=0)
classifier.fit(X_train, y_train) 

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=1000,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [8]:
y_pred = classifier.predict(X_test)

In [9]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))
print(accuracy_score(y_test, y_pred))

[[183  25]
 [ 32 160]]
              precision    recall  f1-score   support

           0       0.85      0.88      0.87       208
           1       0.86      0.83      0.85       192

    accuracy                           0.86       400
   macro avg       0.86      0.86      0.86       400
weighted avg       0.86      0.86      0.86       400

0.8575


In [10]:
with open('text_classifier', 'wb') as picklefile:
    pickle.dump(classifier,picklefile)

In [11]:
import eli5
eli5.show_weights(classifier, top=10)

Using TensorFlow backend.


Weight,Feature
0.0222 ± 0.0536,x102
0.0128 ± 0.0383,x1482
0.0083 ± 0.0276,x144
0.0068 ± 0.0249,x1260
0.0068 ± 0.0221,x893
0.0066 ± 0.0217,x1274
0.0063 ± 0.0214,x98
0.0061 ± 0.0208,x979
0.0061 ± 0.0207,x1081
0.0060 ± 0.0198,x1431


In [12]:
eli5.show_weights(classifier, vec=vectorizer, top=10)

Weight,Feature
0.0222 ± 0.0536,bad
0.0128 ± 0.0383,worst
0.0083 ± 0.0276,boring
0.0068 ± 0.0249,stupid
0.0068 ± 0.0221,nothing
0.0066 ± 0.0217,supposed
0.0063 ± 0.0214,awful
0.0061 ± 0.0208,plot
0.0061 ± 0.0207,ridiculous
0.0060 ± 0.0198,waste


In [18]:
eli5.show_weights(classifier, vec=vectorizer, top=10, target_names=['neg', 'pos'])

Weight,Feature
0.0222 ± 0.0536,bad
0.0128 ± 0.0383,worst
0.0083 ± 0.0276,boring
0.0068 ± 0.0249,stupid
0.0068 ± 0.0221,nothing
0.0066 ± 0.0217,supposed
0.0063 ± 0.0214,awful
0.0061 ± 0.0208,plot
0.0061 ± 0.0207,ridiculous
0.0060 ± 0.0198,waste


In [22]:
doc = documents[413]
import eli5
eli5.explain_prediction(classifier, doc, vec=vectorizer,target_names=['neg','pos'], top=20)

Explanation(estimator="RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',\n                       max_depth=None, max_features='auto', max_leaf_nodes=None,\n                       min_impurity_decrease=0.0, min_impurity_split=None,\n                       min_samples_leaf=1, min_samples_split=2,\n                       min_weight_fraction_leaf=0.0, n_estimators=1000,\n                       n_jobs=None, oob_score=False, random_state=0, verbose=0,\n                       warm_start=False)", description='\nFeatures with largest coefficients.\n\nFeature weights are calculated by following decision paths in trees\nof an ensemble (or a single tree for DecisionTreeClassifier).\nEach node of the tree has an output score, and contribution of a feature\non the decision path is how much the score changes from parent to child.\nWeights of all features sum to the output score or proba of the estimator.\n\nCaveats:\n1. Feature weights just show if the feature contributed positively or\n   negatively to the final score, and does not show how increasing or\n   decreasing the feature value will change the prediction.\n2. In some cases, feature weight can be close to zero for an important feature.\n   For example, in a single tree that computes XOR function, the feature at the\n   top of the tree will have zero weight because expected scores for both\n   branches are equal, so decision at the top feature does not change the\n   expected score. For an ensemble predicting XOR functions it might not be\n   a problem, but it is not reliable if most trees happen to choose the same\n   feature at the top.\n', error=None, method='decision path', is_regression=False, targets=[TargetExplanation(target='pos', feature_weights=FeatureWeights(pos=[FeatureWeight(feature='<BIAS>', weight=0.5049593749999999, std=None, value=1.0), FeatureWeight(feature='perfect', weight=0.02432822221422977, std=None, value=2.0), FeatureWeight(feature='true', weight=0.0189169108506075, std=None, value=1.0), FeatureWeight(feature='life', weight=0.014241433555436873, std=None, value=3.0), FeatureWeight(feature='job', weight=0.011676634594094267, std=None, value=1.0), FeatureWeight(feature='ride', weight=0.009152907135471207, std=None, value=1.0), FeatureWeight(feature='also', weight=0.008197777823420547, std=None, value=1.0), FeatureWeight(feature='plot', weight=0.007694886263318917, std=None, value=0.0), FeatureWeight(feature='george', weight=0.007558899640223573, std=None, value=1.0), FeatureWeight(feature='bill', weight=0.007201737626520927, std=None, value=1.0), FeatureWeight(feature='brings', weight=0.0068975199987536895, std=None, value=1.0), FeatureWeight(feature='best', weight=0.006859411987960527, std=None, value=1.0), FeatureWeight(feature='worst', weight=0.006837858424568377, std=None, value=0.0), FeatureWeight(feature='epic', weight=0.006711709682526635, std=None, value=1.0)], neg=[FeatureWeight(feature='bad', weight=-0.07642658483077648, std=None, value=2.0), FeatureWeight(feature='nothing', weight=-0.03532854524973407, std=None, value=1.0), FeatureWeight(feature='minute', weight=-0.02951499777296064, std=None, value=1.0), FeatureWeight(feature='could', weight=-0.027931187250550975, std=None, value=1.0), FeatureWeight(feature='trying', weight=-0.01082895696875684, std=None, value=1.0), FeatureWeight(feature='every', weight=-0.008139632834081725, std=None, value=1.0)], pos_remaining=872, neg_remaining=575), proba=0.661, score=None, weighted_spans=WeightedSpans(docs_weighted_spans=[DocWeightedSpans(document='susan granger review of the perfect storm warner bros more people die on fishing boats per capita than working in any other job in the s every journey fishing boat makes can be an all or nothing risk it is life at its most exhilarating and its most terrifying says director wolfgang petersen das boot and that just what he captures in this true story of struggle and humanity aboard swordfishing boat the andrea gail sailing out of gloucester m

In [23]:
eli5.show_prediction(classifier, doc, target_names=['neg', 'pos'], vec=vectorizer, top=20)